# Setup

### Installing external libraries

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs
# for using huggingface in kaggle
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
# for evaluation
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-26rtr86u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-26rtr86u
  Resolved https://github.com/huggingface/transformers.git to commit 2489e380e45177eb3da108366f65b8108b2815c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7344228 sha256=aff20e8e80329a7008e01c8cdf0b935365f3171262aadfc8c2b24403fc1dd67e
  Stored in directory: /tmp/pip-ephem-wheel-cache-p57luo2w/wheels/

### setting up hugginface hub account for using inference (Pipeline)

In [2]:
!git config --global user.email "imtiazahmed5573@gmail.com" # put your huggingface account mail here
!git config --global user.name "imtiaz114" # put your huggingface account user name here

In [3]:
# Enter your code by tapping on the "Hugging Face tokens" portion of generated message
from huggingface_hub import notebook_login
notebook_login()

### Importing necessary libraries

In [4]:
# basic libraries
import pandas as pd
import numpy as np
import itertools
import gc

# evaluation
import evaluate

# dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Tokenizer
from transformers import AutoTokenizer
# collat fn
from transformers import DataCollatorForTokenClassification
# model
from transformers import TFAutoModelForTokenClassification
# callback (useful for using pipeline function for inference)
from transformers.keras_callbacks import PushToHubCallback

# prediction
from transformers import pipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
# paths
model_checkpoint = "sagorsarker/mbert-bengali-ner"
dirname = "/kaggle/working/mbert-ner-baseline-1" # model-saving-directory

dataset_path = "/kaggle/input/hishab-dataset-person-extractor/concatened-augmented-ner-bangla-ds/concatened-augmented-ner-bangla-ds.csv"

In [6]:
# Hyperparamters
num_epochs = 4 #4 # 2 or 4 epochs are suitable for this task
INIT_LR    = 2e-5
WD         = 0.01

# Dataset

### Reading the dataset

The data here was prepared in another notebook named " ", please refer that to get an idea about data preparation

In [7]:
df = pd.read_csv(dataset_path,index_col=False)
df.head(5)

,tokens,tags,flag
0,"['\\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাং...","['O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O...",0
1,"['অর্থনীতি', 'ও', 'আর্থসামাজিক', 'বেশির', 'ভাগ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '...",0
2,"['নিম্ন', 'আয়ের', 'দেশগুলোকে', 'ছাড়িয়েছে', 'তো...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",0
3,"['সবাইকে', 'অন্তর্ভুক্ত', 'করে', 'প্রবৃদ্ধি', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '...",0
4,"['তবে', 'অন্তর্ভুক্তিমূলক', 'প্রবৃদ্ধি', 'অর্জ...","['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', '...",0


In [8]:
# taking dataframe columns into lists
tokens = df["tokens"].tolist()
tags   = df["tags"].tolist()
print(len(tokens),len(tags))

23889 23889


Now, these tokens and tags are in form of strings (whole sentences), we need to extract individual tokens and tags from it. So, I am extracting tokens from these sentences some punctuation that was causing error in the tokenization process

In [9]:
# tags and tokens fixing
newTags = []
for idx in range(len(tags)):
    demo = tags[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")

    newTags.append(demo)
    
tags = newTags
#############
newTokens = []
for idx in range(len(tokens)):
    demo = tokens[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")

    newTokens.append(demo)
tokens = newTokens
print(len(tokens),len(tags))

23889 23889


In [10]:
# checking 
print(tokens[0][0])
print(tags[0][0])

\\ufeffউন্নয়নের
O


now I am checking the unique label names for the encoding process coming next

In [11]:
flattened_list = list(itertools.chain(*tags))
label_names = list(set(flattened_list))
label_names# checking unique tag names

['B-ORG', 'I-LOC', 'B-PER', 'I-PER', 'O', 'I-ORG', 'B-LOC']

In [12]:
# shuffling to a suitable form for later
label_name = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
label_name

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

### converting tags to integers

In [13]:
prevtags = tags[:3]
for i in range(len(tags)):
    demo =tags[i]
    for j in range(len(demo)):
        demo[j] = label_name.index(demo[j])
    tags[i] = demo

for j in range(3):
    print(prevtags[j])
    print("\n")
    print(tags[i])

[0, 0, 5, 5, 0, 0, 0, 0, 0]


[4, 0, 4, 4, 4, 4, 0, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 6, 0]


[4, 0, 4, 4, 4, 4, 0, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 0, 0, 0, 0, 0, 0]


[4, 0, 4, 4, 4, 4, 0, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4]


In [14]:
# checking alignment of token and tags
words = tokens[0]
labels = tags[0]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_name[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

\\ufeffউন্নয়নের বিস্ময় বাংলাদেশ বাংলাদেশের অগ্রগতি উদাহরণ দেওয়ার মতোই । 
O               O      B-LOC    B-LOC      O       O      O      O    O 


## Tokenizing (for the model)

* using pretrained tokenizer suitable for our model

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [16]:
# checking tokenizer at play, it will add additional level for the understanding of the model
inputs = tokenizer(tokens[0], is_split_into_words=True)
print(inputs.tokens())
print(inputs.word_ids()) # priniting word id associated with this

['[CLS]', '\\', '\\', 'u', '##fe', '##ff', '##উ', '##ন', '##ন', '##যন', '##ের', 'ব', '##িস', '##ময', 'বাংলাদেশ', 'বাংলাদেশের', 'অ', '##গর', '##গত', '##ি', 'উ', '##দা', '##হ', '##রণ', 'দেওযা', '##র', 'মতো', '##ই', '।', '[SEP]']
[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 8, None]


In [17]:
def align_labels_with_tokens(labels, word_ids):
    """
        This function aligns labels and word_ids
    """
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
# checking it
labels = tags[0]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 5, 5, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


## Creating datasets

In [19]:
def cleaning_data(tokens, tags):
    """
        This function deletes all the mismatched token-tag pairs and generate flags which will be used
        for splitting the dataset
    """
    mismatch = []
    newtokens = []
    newtags   = []
    flags      = []
    newcount = 0
    for i in range(len(tokens)):
        c = 0
        if len(tokens[i]) > 0 and len(tags[i])>0:
            if len(tokens[i]) != len(tags[i]):
                mismatch.append(i)
            else:
                newtokens.append(tokens[i])
                newtags.append(tags[i])
                
                demo = tags[i]
                for j in demo:
                    if j == 1 or j == 2:
                        c = 1
                if c == 1:
                    flags.append(1)
                else:
                    flags.append(0)
                

                
    print("total number of flag sentences %d"%(sum(flags)))
    print(len(flags),"\n")
    print("total number of mismatched token and tag pair sentneces %d"%(len(mismatch)))
    print("total number of sentneces %d"%(i+1))
    print("final length of dataset %d"%len(newtokens))
    return newtokens,newtags,flags

tokens,tags,flags = cleaning_data(tokens, tags)

total number of flag sentences 6135
23889 

total number of mismatched token and tag pair sentneces 0
total number of sentneces 23889
final length of dataset 23889


### splitting the dataset

In [20]:
# Split the data into train, validation, and test datasets
tokens_train, tokens_val, tags_train, tags_val = train_test_split(tokens, tags,stratify=flags, test_size=0.2, random_state=42)
#tokens_val, tokens_test, tags_val, tags_test = train_test_split(tokens_val, tags_val, test_size=0.25, random_state=42)

# Print the split datasets
print("Train dataset:")
print(tokens_train[0])
print(tags_train[0])
print(len(tokens_train),"\n")

print("Validation dataset:")
print(tokens_val[0])
print(tags_val[0])
print(len(tokens_val),"\n")

#print("Test dataset:")
#print(tokens_test[0])
#print(tags_test[0])
#print(len(tokens_test),"\n")

Train dataset:
['ছিল', 'মধ্যেও', 'জয়', 'প্লাজার', 'দুর্ঘটনায়', 'দৃষ্টান্ত', 'দুঃখের', 'মানবতার', 'অনন্য', 'রানা']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
19111 

Validation dataset:
['আইনে', 'বিবাহ', 'তবে', 'হলে', 'নন', 'অনেকে', 'অবগত', 'আছে', 'সে', '<PUNCT>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4778 



In [21]:
gc.collect()

186

## preparing the dataset

In [22]:
# Create Dataset objects for train, validation, and test splits
train_dataset = Dataset.from_dict({'tokens': tokens_train, 'tags': tags_train})
validation_dataset = Dataset.from_dict({'tokens': tokens_val, 'tags': tags_val})
#test_dataset = Dataset.from_dict({'tokens': tokens_test, 'tags': tags_test})

# Create a DatasetDict object
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    #'test': test_dataset
})

In [23]:
raw_datasets["train"].column_names

['tokens', 'tags']

In [24]:
def tokenize_and_align_labels(examples):
    """
        this function will tokenize using pretrained tokenizer and align labels and word ids
    """
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [25]:
# mapping the datasets
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [26]:
# collat function for preparing dataset
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [27]:
# checking on a batch
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 31), dtype=int64, numpy=
array([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    5,    6, -100],
       [-100,    0,    0,    0,    0,    1,    2,    2,    2,    2,    0,
           0,    0,    5,    6,    6,    6,    6,    6,    0,    0,    0,
           0,    0,    0,    0, -100, -100, -100, -100, -100]])>

In [28]:
# finalizing the datasets
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,#16
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,#16
)

In [29]:
# creating mapping dictionary for ease of mapping
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [30]:
gc.collect()

46

# Training

## Building the model

In [31]:
# Initialize the model
model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [32]:
# checking number of classes with the model
model.config.num_labels

7

## compiling the model

In [33]:
from transformers import create_optimizer
import tensorflow as tf

# mixed precision (mainly needed if ran in CPU)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=INIT_LR,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=WD,
)
model.compile(optimizer=optimizer)

## Training the model

In [34]:
callback = PushToHubCallback(output_dir=dirname, tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

Cloning https://huggingface.co/imtiaz114/mbert-ner-baseline-1 into local empty directory.


Download file tf_model.h5:   0%|          | 16.5k/636M [00:00<?, ?B/s]

Download file spiece.model:   0%|          | 8.00k/5.38M [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 8.00k/2.44M [00:00<?, ?B/s]

Download file sentencepiece.bpe.model:   0%|          | 24.0k/4.83M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/2.44M [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/5.38M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/636M [00:00<?, ?B/s]

Epoch 1/4
2388/2388 [==============================] - 443s 175ms/step - loss: 0.2646 - val_loss: 0.1872
Epoch 2/4
2388/2388 [==============================] - 446s 187ms/step - loss: 0.1604 - val_loss: 0.1713
Epoch 3/4
2388/2388 [==============================] - 447s 187ms/step - loss: 0.1127 - val_loss: 0.1295
Epoch 4/4
2388/2388 [==============================] - 449s 188ms/step - loss: 0.0834 - val_loss: 0.1248


# Evaluation

In [35]:
metric = evaluate.load("seqeval")

In [36]:
all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
# computing classification metrics
metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'precision': 0.8089646046973206,
  'recall': 0.8387926599211113,
  'f1': 0.8236086553843562,
  'number': 5831},
 'ORG': {'precision': 0.9753502755118922,
  'recall': 0.9745317685353542,
  'f1': 0.9749408502302389,
  'number': 157294},
 'PER': {'precision': 0.8635198633062794,
  'recall': 0.8358486665288402,
  'f1': 0.8494589767832755,
  'number': 9674},
 'overall_precision': 0.963452935586306,
 'overall_recall': 0.9621872811763957,
 'overall_f1': 0.9628196924466733,
 'overall_accuracy': 0.9561988234142734}

# Prediction (on sentences)

In [37]:
model_checkpoint = dirname
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

text = "প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।"
token_classifier(text)

Some layers from the model checkpoint at /kaggle/working/mbert-ner-baseline-1 were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at /kaggle/working/mbert-ner-baseline-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[{'entity_group': 'ORG', 'score': 1.0, 'word': 'পরতি', 'start': 0, 'end': 5},
 {'entity_group': 'ORG', 'score': 0.999, 'word': '##বে', 'start': 5, 'end': 7},
 {'entity_group': 'ORG', 'score': 0.999, 'word': '##দ', 'start': 7, 'end': 8},
 {'entity_group': 'ORG', 'score': 1.0, 'word': '##ক', 'start': 8, 'end': 9},
 {'entity_group': 'ORG',
  'score': 1.0,
  'word': 'ছিলেন',
  'start': 10,
  'end': 15},
 {'entity_group': 'ORG', 'score': 0.998, 'word': 'স', 'start': 16, 'end': 17},
 {'entity_group': 'ORG',
  'score': 0.996,
  'word': '##াপ',
  'start': 17,
  'end': 19},
 {'entity_group': 'ORG',
  'score': 0.997,
  'word': '##তা',
  'start': 20,
  'end': 22},
 {'entity_group': 'ORG',
  'score': 0.997,
  'word': '##হ',
  'start': 22,
  'end': 23},
 {'entity_group': 'ORG',
  'score': 0.996,
  'word': '##িক',
  'start': 23,
  'end': 25},
 {'entity_group': 'ORG',
  'score': 0.9863,
  'word': '##ের র',
  'start': 25,
  'end': 29},
 {'entity_group': 'LOC',
  'score': 0.966,
  'word': '##াজ',
  'st

In [38]:
def gettingPersonName(inputs):
    """
        This function takes inputs a sentence and gives persons name output
    """
    print(inputs)
    output = token_classifier(inputs)
    
    
    person_names = []
    current_person = ""

    for entity in output:
        if entity['entity_group'] == 'PER':
            current_person += entity['word']
            person_names.append(current_person)
            current_person = ""

        else:
            if current_person:
                current_person = ""

    # Add the last person name entity if present
    if current_person:
        person_names.append(current_person)
        
    clean_names=[]
    for demo in person_names:
        demo = demo.replace("#", "" )
        clean_names.append(demo)
    
    if len(clean_names)>0:
        for names in clean_names:
            print(names)
    else:
        print("There is no person in this sentence")
        
    return clean_names

text = "প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।"
names = gettingPersonName(text)
names

প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।
ায
ে
ম
স
াব


['ায', 'ে', 'ম', 'স', 'াব']

In [39]:
text =  "নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।"
names = gettingPersonName(text)
names

নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।
ান
মে
াহ
ামমদ
মে
া
শা
র
র
ফ
ের


['ান', 'মে', 'াহ', 'ামমদ', 'মে', 'া', 'শা', 'র', 'র', 'ফ', 'ের']

In [40]:
text = "ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।"
names = gettingPersonName(text)
names

ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।
There is no person in this sentence


[]

In [41]:
text = "প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।"
names = gettingPersonName(text)
names

প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।
যার
ো
লি
ন
ও
জন
িযা
কি
রি
ম
যা
ক
ই
ল
র
যের


['যার',
 'ো',
 'লি',
 'ন',
 'ও',
 'জন',
 'িযা',
 'কি',
 'রি',
 'ম',
 'যা',
 'ক',
 'ই',
 'ল',
 'র',
 'যের']

In [42]:
text = "আমার সোনার বাংলা আমি তোমায় ভালবাসি"
names = gettingPersonName(text)

আমার সোনার বাংলা আমি তোমায় ভালবাসি
There is no person in this sentence


# Conclusion